In [ ]:
from datasets import Dataset, DatasetDict
import torch
from random import randrange, sample
from transformers import DataCollatorForSeq2Seq, T5ForConditionalGeneration
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, PeftModel, PeftConfig
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support
import json
import argparse
import tqdm
import numpy as np
import random
import os
from skllm.config import SKLLMConfig
from skllm import MultiLabelZeroShotGPTClassifier

In [ ]:
train_data = pd.read_csv('./synthetic_data/Iteration_1.csv')
test_data = pd.read_csv('./synthetic_data/Partial_Iteration_2_annotated.csv')

In [ ]:
# drop rows where column label is empty
test_data = test_data.dropna(subset=['label'])

Remove legacy categories

In [ ]:
test_data['label'] = test_data['label'].str.split(',')

# Remove 'CAREGIVER' and 'EDUCATION' from the label list
test_data['label'] = test_data['label'].apply(lambda x: [label.strip() for label in x if label.strip() not in ['CAREGIVER', 'EDUCATION']])

# Convert the label list back to a comma-separated string
test_data['label'] = test_data['label'].apply(lambda x: ','.join(x))

test_data = test_data[test_data['label'] != '']

In [ ]:
test_text = test_data['text'].tolist()
test_labels = test_data['label'].tolist()
testdf = pd.DataFrame({'text':test_text, 'SDOHlabels':test_labels})
test_dataset = Dataset.from_pandas(testdf)

In [ ]:
train_text = train_data['text'].tolist()
train_labels = train_data['label'].tolist()
traindf = pd.DataFrame({'text':train_text, 'SDOHlabels':train_labels})
train_dataset = Dataset.from_pandas(traindf)

In [ ]:
BROAD_LABELS = {'TRANSPORTATION', 'HOUSING', 'RELATIONSHIP',
                'PARENT','EMPLOYMENT', 'SUPPORT'}

## Fine Tuned T5

In [ ]:
model_path = 'path/to/finetuned/t5'

In [ ]:
TOKENIZER = AutoTokenizer.from_pretrained(model_path)
MAX_S_LEN = 100
MAX_T_LEN = 40

In [ ]:
config = PeftConfig.from_pretrained(model_path)

# load base LLM model and tokenizer
reloaded_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})

## tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# Load the Lora model
reloaded_model = PeftModel.from_pretrained(reloaded_model, model_path, device_map={"":0})
reloaded_model.eval()

Support functions

In [ ]:
def postprocess_function(preds):
    """
    Perform post-processing on the predictions.

    Args:
        preds (list): A list of predictions.

    Returns:
        list: Processed predictions with fixed labels.

    Examples:
        >>> preds = ['REL', 'EMPLO', 'HOUS', 'UNKNOWN']
        >>> postprocess_function(preds)
        ['RELATIONSHIP', 'EMPLOYMENT', 'HOUSING', 'UNKNOWN']

        >>> preds = ['NO_SD', np.nan, 'SUPP']
        >>> postprocess_function(preds)
        ['<NO_SDOH>', '<NO_SDOH>', 'SUPPORT']
    """
    lab_fixed_dict = {
        'REL': 'RELATIONSHIP',
        'RELAT': 'RELATIONSHIP',
        'EMP': 'EMPLOYMENT',
        'EMPLO': 'EMPLOYMENT',
        'SUPP': 'SUPPORT',
        'HOUS': 'HOUSING',
        'PAREN': 'PARENT',
        'TRANSPORT': 'TRANSPORTATION',
        'NO_SD': '<NO_SDOH>',
        np.nan: '<NO_SDOH>',
        'NO_SDOH>': '<NO_SDOH>',
        '<NO_SDOH': '<NO_SDOH>',
    }

    new_preds = []
    for pred in preds:
        pred_ls = []
        pred = str(pred)
        for pp in pred.split(','):
            if pp in lab_fixed_dict.keys():
                pred_ls.append(lab_fixed_dict[pp])
            else:
                pred_ls.append(pp)
        new_preds.append(','.join(pred_ls))

    return new_preds

def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["text"]]
    # tokenize inputs
    model_inputs = TOKENIZER(inputs, max_length=MAX_S_LEN, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = TOKENIZER(text_target=sample["SDOHlabels"], max_length=MAX_T_LEN, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != TOKENIZER.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def normal_eval(preds, gold):
    pred_temp = [p.split(",") for p in preds]
    gold_list = [g.split(',') for g in gold]

    pred_list = []
    for labs in pred_temp:
        point_pred = [p for p in labs if p in BROAD_LABELS]
        pred_list.append(point_pred)
    mlb = MultiLabelBinarizer()
    oh_gold = mlb.fit_transform(gold_list)
    oh_pred = mlb.transform(pred_list)

    prec, rec, f1, _ = precision_recall_fscore_support(oh_gold, oh_pred)
    micro_f1  = precision_recall_fscore_support(oh_gold, oh_pred, average='micro')[2]
    weight_f1 = precision_recall_fscore_support(oh_gold, oh_pred, average='weighted')[2]
    macro_f1 = precision_recall_fscore_support(oh_gold, oh_pred, average='macro')[2]

    metrics_out = {'macro_f1':macro_f1, 'micro_f1': micro_f1, 'weighted_f1': weight_f1}
    for i, lab in enumerate(list(mlb.classes_)):
        metrics_out['precision_'+str(lab)] = prec[i]
        metrics_out['recall_'+str(lab)] = rec[i]
        metrics_out['f1_'+str(lab)] = f1[i]
    print(classification_report(oh_gold, oh_pred, target_names=mlb.classes_))
    return metrics_out

def predict(dataset, model, batch_size):
    predictions, references = [], []
    batch_size = batch_size
    for i in tqdm.tqdm(range(0, len(dataset), batch_size)):
        texts = dataset[i:i+batch_size]
        input_ids = TOKENIZER(texts["text"], return_tensors="pt", truncation=True, padding="max_length").input_ids.cuda()
        outputs = model.generate(input_ids=input_ids, do_sample=False, top_p=0.9, max_new_tokens=5, num_beams=4) #, top_p=0.9, max_new_tokens=10
        outputs = TOKENIZER.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True) 
        labels = dataset[i:i+batch_size]["SDOHlabels"] 
        predictions.extend(outputs)
        references.extend(labels)
    return predictions, references

In [ ]:
predictions, references = predict(test_dataset, reloaded_model, 4)
metrics = normal_eval(predictions, references)
print('='*30+'POST PROCESSED'+'='*30)
processed_predictions = postprocess_function(predictions)
processed_metrics = normal_eval(processed_predictions, references)

## GPT

In [ ]:
SKLLMConfig.set_openai_key("API KEY HERE")
SKLLMConfig.set_openai_org("ORGANIZATION HERE")

#### Multi-Label

In [ ]:
clf = MultiLabelZeroShotGPTClassifier(max_labels=4)
clf.fit(None, [BROAD_LABELS])

In [ ]:
labels = clf.predict(test_data['text'])
y = [foo.split(',') for foo in test_data['label']]

In [ ]:
mlb2 = MultiLabelBinarizer()
y = mlb2.fit_transform(y)
labels = mlb2.transform(labels)

In [ ]:
print(classification_report(y, labels, target_names=mlb2.classes_))